In [ ]:
# Install the latest version of crossdeploy
! pip install --upgrade crossdeploy

# Node is installed by default in Watson Studio, JupyterLab and JupyterNotebook environments. 
# However, in case your environment does not have node installed, it can be installed with the below command.
# ! pip install "nodejs-bin[cmd]"

# Example 1: Understanding how declarative approach works

There are 3 parts to do this example,

- Part A: Running the same code repeatly

- Part B: Modifying the model name

- Part C: Modifying the model's hyperparameter


# Imports

In [ ]:
import pandas as pd

from crossdeploy.utils import utils
from crossdeploy.crossdeploy import CrossDeploy

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_selector, make_column_transformer

# Define variables

In [ ]:
CPD_API_KEY = "xxx" # or CPD_PASSWORD = "xxx"
CPD_USERNAME = "admin"
CPD_URL = "https://cpd-cpd.itzroks-550003aw18-bwcq3a-6ccd7f378ae819553d37d5f2ee142bd6-0000.au-syd.containers.appdomain.cloud"
PROJECT_ID = "75c938f9-cc5f-45a9-b44f-3f1b51a69516"

# Import data and preprocessing step

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/crossdeploy-io/crossdeploy-examples/main/data/mortgage-default.csv")

label_column = "MortgageDefault"
y = df[label_column]
X = df.drop(label_column, axis=1)

ct = make_column_transformer(
    (OneHotEncoder(), make_column_selector(dtype_include=object)),
    remainder="passthrough"
)

# Part A: Running the same code repeatedly

Running the code below repeatedly, will result in one model being created in your project.

Note that the model id, always remains the same.

In [ ]:
MODEL_NAME = "mortgage-model-rf"

model = Pipeline(steps=[
    ("transform", ct), 
    ("clf", RandomForestClassifier(n_estimators=3, max_depth=3, random_state=12345))
])
model.fit(X, y)
model.score(X, y)

flow = CrossDeploy()

flow.ibm.Provider(url=CPD_URL, username=CPD_USERNAME, api_key=CPD_API_KEY) # or password=CPD_PASSWORD

model_rf = flow.ibm.Model(model)(
    id = "mortgage-model-rf",
    name = MODEL_NAME,
    project_id = PROJECT_ID,
)

flow.apply()

model_state = utils.get_resource_by_name(model_rf.friendly_unique_id)

print()
print(f"Model ID: {model_state['id']}")

# Part B: Modifying the model name

Try modifying the `MODEL_NAME` and running the code.

You should see that it only results in an update instead of recreating a model.

In addition, the model id remains the same as the earlier example.

In [ ]:
MODEL_NAME = "mortgage-model"

model = Pipeline(steps=[
    ("transform", ct), 
    ("clf", RandomForestClassifier(n_estimators=3, max_depth=3, random_state=12345))
])
model.fit(X, y)
model.score(X, y)

flow = CrossDeploy()

flow.ibm.Provider(url=CPD_URL, username=CPD_USERNAME, api_key=CPD_API_KEY)

model_rf = flow.ibm.Model(model)(
    id = "mortgage-model-rf",
    name = MODEL_NAME,
    project_id = PROJECT_ID,
)

flow.apply()

model_state = utils.get_resource_by_name(model_rf.friendly_unique_id)

print()
print(f"Model ID:   {model_state['id']}")
print(f"Model Name: {model_state['name']}")

# Part C: Modifying the model's hyperparameter

Change`n_estimators` or `max_depth` to another value.

This is basically a new version of the model as training the model with new hyperparameters is a significant change.

In this case, you should see that the model is destroyed and recreated. A new model id will be produced.

In [ ]:
MODEL_NAME = "mortgage-model-rf"

model = Pipeline(steps=[
    ("transform", ct), 
    ("clf", RandomForestClassifier(n_estimators=3, max_depth=10, random_state=12345))
])
model.fit(X, y)
model.score(X, y)

flow = CrossDeploy()

flow.ibm.Provider(url=CPD_URL, username=CPD_USERNAME, api_key=CPD_API_KEY)

model_rf = flow.ibm.Model(model)(
    id = "mortgage-model-rf",
    name = MODEL_NAME,
    project_id = PROJECT_ID,
)

flow.apply()

model_state = utils.get_resource_by_name(model_rf.friendly_unique_id)

print()
print(f"Model ID: {model_state['id']}")

# Clean up

In [ ]:
flow.destroy()